In [6]:
import requests
from requests import Session

from base64 import b64encode, b64decode
import hashlib
import sys
import crypto
sys.modules['Crypto'] = crypto
sys.path.append(r'/c/Users/Go/Desktop/TapoP100-main/PyP100/')


from Crypto.PublicKey import RSA
import time
import json
from Crypto.Cipher import AES, PKCS1_OAEP, PKCS1_v1_5
import tp_link_cipher
import ast
import pkgutil
import uuid
import json

ERROR_CODES = {
	"0": "Success",
	"-1010": "Invalid Public Key Length",
	"-1012": "Invalid terminalUUID",
	"-1501": "Invalid Request or Credentials",
	"1002": "Incorrect Request",
	"-1003": "JSON formatting error "
}

In [18]:
import requests
from requests import Session

from base64 import b64encode, b64decode
import hashlib
import sys
import crypto
sys.modules['Crypto'] = crypto
sys.path.append(r'/c/Users/Go/Desktop/TapoP100-main/PyP100/')


from Crypto.PublicKey import RSA
import time
import json
from Crypto.Cipher import AES, PKCS1_OAEP, PKCS1_v1_5
import tp_link_cipher
import ast
import pkgutil
import uuid
import json


#Old Functions to get device list from tplinkcloud
def getToken(email, password):
	URL = "https://eu-wap.tplinkcloud.com"
	Payload = {
		"method": "login",
		"params": {
			"appType": "Tapo_Ios",
			"cloudUserName": email,
			"cloudPassword": password,
			"terminalUUID": "0A950402-7224-46EB-A450-7362CDB902A2"
		}
	}

	return requests.post(URL, json=Payload).json()['result']['token']


def getDeviceList(email, password):
	URL = "https://eu-wap.tplinkcloud.com?token=" + getToken(email, password)
	Payload = {
		"method": "getDeviceList",
	}

	return requests.post(URL, json=Payload).json()

ERROR_CODES = {
	"0": "Success",
	"-1010": "Invalid Public Key Length",
	"-1012": "Invalid terminalUUID",
	"-1501": "Invalid Request or Credentials",
	"1002": "Incorrect Request",
	"-1003": "JSON formatting error "
}

class P100():
	def __init__ (self, ipAddress, email, password):
		self.ipAddress = ipAddress
		self.terminalUUID = str(uuid.uuid4())

		self.email = email
		self.password = password
		self.session = Session()

		self.errorCodes = ERROR_CODES

		self.encryptCredentials(email, password)
		self.createKeyPair()

	def encryptCredentials(self, email, password):
		#Password Encoding
		self.encodedPassword = tp_link_cipher.TpLinkCipher.mime_encoder(password.encode("utf-8"))

		#Email Encoding
		self.encodedEmail = self.sha_digest_username(email)
		self.encodedEmail = tp_link_cipher.TpLinkCipher.mime_encoder(self.encodedEmail.encode("utf-8"))

	def createKeyPair(self):
		self.keys = RSA.generate(1024)

		self.privateKey = self.keys.exportKey("PEM")
		self.publicKey  = self.keys.publickey().exportKey("PEM")

	def decode_handshake_key(self, key):
		decode: bytes = b64decode(key.encode("UTF-8"))
		decode2: bytes = self.privateKey

		cipher = PKCS1_v1_5.new(RSA.importKey(decode2))
		do_final = cipher.decrypt(decode, None)
		if do_final is None:
			raise ValueError("Decryption failed!")

		b_arr:bytearray = bytearray()
		b_arr2:bytearray = bytearray()

		for i in range(0, 16):
			b_arr.insert(i, do_final[i])
		for i in range(0, 16):
			b_arr2.insert(i, do_final[i + 16])

		return tp_link_cipher.TpLinkCipher(b_arr, b_arr2)

	def sha_digest_username(self, data):
		b_arr = data.encode("UTF-8")
		digest = hashlib.sha1(b_arr).digest()

		sb = ""
		for i in range(0, len(digest)):
			b = digest[i]
			hex_string = hex(b & 255).replace("0x", "")
			if len(hex_string) == 1:
				sb += "0"
				sb += hex_string
			else:
				sb += hex_string

		return sb

	def handshake(self):
		URL = f"http://{self.ipAddress}/app"
		Payload = {
			"method":"handshake",
			"params":{
				"key": self.publicKey.decode("utf-8"),
				"requestTimeMils": int(round(time.time() * 1000))
			}
		}

		r = self.session.post(URL, json=Payload, timeout=2)

		encryptedKey = r.json()["result"]["key"]
		self.tpLinkCipher = self.decode_handshake_key(encryptedKey)

		try:
			self.cookie = r.headers["Set-Cookie"][:-13]

		except:
			errorCode = r.json()["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def login(self):
		URL = f"http://{self.ipAddress}/app"
		Payload = {
			"method":"login_device",
			"params":{
				"username": self.encodedEmail,
				"password": self.encodedPassword
			},
			"requestTimeMils": int(round(time.time() * 1000)),
		}
		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method":"securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		try:
			self.token = ast.literal_eval(decryptedResponse)["result"]["token"]
		except:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOn(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "set_device_info",
			"params":{
				"device_on": True
			},
			"requestTimeMils": int(round(time.time() * 1000)),
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOff(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "set_device_info",
			"params":{
				"device_on": False
			},
			"requestTimeMils": int(round(time.time() * 1000)),
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def getDeviceInfo(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "get_device_info",
			"requestTimeMils": int(round(time.time() * 1000)),
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method":"securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)
		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		return json.loads(decryptedResponse)

	def getDeviceName(self):
		self.handshake()
		self.login()
		data = self.getDeviceInfo()

		if data["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")
		else:
			encodedName = data["result"]["nickname"]
			name = b64decode(encodedName)
			return name.decode("utf-8")

	def turnOnWithDelay(self, delay):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "add_countdown_rule",
			"params": {
				"delay": int(delay),
				"desired_states": {
					"on": True
				},
				"enable": True,
				"remain": int(delay)
			},
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOffWithDelay(self, delay):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "add_countdown_rule",
			"params": {
				"delay": int(delay),
				"desired_states": {
					"on": False
				},
				"enable": True,
				"remain": int(delay)
			},
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")


In [13]:
getToken('rhqudrjs2@naver.com','qudrjs12#')

'bd3eedca-BT4wglWiWgIMD5xWhNcHGht'

In [20]:
getDeviceList('rhqudrjs2@naver.com','qudrjs12#')

{'error_code': 0,
 'result': {'deviceList': [{'deviceType': 'SMART.TAPOPLUG',
    'role': 0,
    'fwVer': '1.4.10 Build 20211104 Rel. 35882',
    'appServerUrl': 'https://aps1-wap.tplinkcloud.com',
    'deviceRegion': 'ap-southeast-1',
    'deviceId': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
    'deviceName': 'P100',
    'deviceHwVer': '1.20.0',
    'alias': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
    'deviceMac': '5CA6E63AB803',
    'oemId': 'D7C5ADFD37188938BA82FD0FE208D1E7',
    'deviceModel': 'P100',
    'hwId': '9994A0A7D5B29645B8150C392284029D',
    'fwId': '1D18AD293A25ABDE41405B20C6F98816',
    'isSameRegion': False,
    'status': 0}]}}

In [21]:
testP100 = P100('192.168.0.115','rhqudrjs2@naver.com','qudrjs12#')

In [24]:
testP100.login()

In [42]:
# testP100.turnOn()
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': True,
  'on_time': 21,
  'overheated': False,
  'nickname': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
  'location': 'office',
  'avatar': 'plug',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.115',
  'ssid': 'RUhSTkNfRFNfMkc=',
  'signal_level': 2,
  'rssi': -54,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': 'ko_KR',
  'default_states': {'type': 'last_states', 'state': {}}}}

In [43]:
testP100.turnOff()
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': False,
  'on_time': 0,
  'overheated': False,
  'nickname': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
  'location': 'office',
  'avatar': 'plug',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.115',
  'ssid': 'RUhSTkNfRFNfMkc=',
  'signal_level': 2,
  'rssi': -54,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': 'ko_KR',
  'default_states': {'type': 'last_states', 'state': {}}}}

In [48]:
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': False,
  'on_time': 0,
  'overheated': False,
  'nickname': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
  'location': 'office',
  'avatar': 'plug',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.115',
  'ssid': 'RUhSTkNfRFNfMkc=',
  'signal_level': 2,
  'rssi': -53,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': 'ko_KR',
  'default_states': {'type': 'last_states', 'state': {}}}}